In [1]:
#Import das bibliotecas necessárias

import requests
import zipfile
import io
import pandas as pd
import numpy as np
import os

In [2]:
# 1. Configuração dos Caminhos
# URL direta do dataset na UCI
url = "https://archive.ics.uci.edu/static/public/447/condition+monitoring+of+hydraulic+systems.zip"
extract_path = "../data/raw/hydraulic"
processed_path = "../data/processed"

os.makedirs(extract_path, exist_ok=True)
os.makedirs(processed_path, exist_ok=True)

print("1. Baixando dataset (pode demorar um pouco)...")
response = requests.get(url)
z = zipfile.ZipFile(io.BytesIO(response.content))
z.extractall(extract_path)
print("   Download e extração concluídos.")

1. Baixando dataset (pode demorar um pouco)...
   Download e extração concluídos.


In [3]:
# 2. Definição dos Sensores
# Cada arquivo .txt é um sensor. Vamos ler e tirar a média de cada ciclo.
sensores = {
    'PS1': 'Pressure_1', 'PS2': 'Pressure_2', 'PS3': 'Pressure_3', 
    'PS4': 'Pressure_4', 'PS5': 'Pressure_5', 'PS6': 'Pressure_6',
    'FS1': 'Volume_Flow_1', 'FS2': 'Volume_Flow_2',
    'TS1': 'Temperature_1', 'TS2': 'Temperature_2', 'TS3': 'Temperature_3', 'TS4': 'Temperature_4',
    'VS1': 'Vibration', 'CE': 'Cooling_Efficiency', 'CP': 'Cooling_Power', 'SE': 'Efficiency_Factor'
}

df_final = pd.DataFrame()

print("2. Processando sensores (agregando séries temporais)...")
for filename, col_name in sensores.items():
    file_path = os.path.join(extract_path, f'{filename}.txt')
    
    # Lendo o arquivo (separado por tabs, sem cabeçalho)
    # Cada linha é um ciclo de 60 segundos. Vamos pegar a média da linha.
    data = pd.read_csv(file_path, sep='\t', header=None)
    
    # Feature Engineering simples: Média do ciclo
    df_final[col_name] = data.mean(axis=1)
    print(f"   -> Sensor {col_name} processado.")

2. Processando sensores (agregando séries temporais)...
   -> Sensor Pressure_1 processado.
   -> Sensor Pressure_2 processado.
   -> Sensor Pressure_3 processado.
   -> Sensor Pressure_4 processado.
   -> Sensor Pressure_5 processado.
   -> Sensor Pressure_6 processado.
   -> Sensor Volume_Flow_1 processado.
   -> Sensor Volume_Flow_2 processado.
   -> Sensor Temperature_1 processado.
   -> Sensor Temperature_2 processado.
   -> Sensor Temperature_3 processado.
   -> Sensor Temperature_4 processado.
   -> Sensor Vibration processado.
   -> Sensor Cooling_Efficiency processado.
   -> Sensor Cooling_Power processado.
   -> Sensor Efficiency_Factor processado.


In [4]:
# 3. Processando o Target (O que queremos prever)
# O arquivo profile.txt tem os estados dos componentes.
# Coluna 4 (índice 4 no python? nao, é a ultima) é o "Stable Flag" (0 = Estável, 1 = Instável)
print("3. Processando Target (Estabilidade)...")
profile = pd.read_csv(os.path.join(extract_path, 'profile.txt'), sep='\t', header=None)

# A coluna 4 indica estabilidade: 0 = estável, 1 = instável
df_final['Stable_Flag'] = profile[4]

3. Processando Target (Estabilidade)...


In [5]:
# 4. Salvando
output_file = os.path.join(processed_path, 'hydraulic_uci_processed.csv')
df_final.to_csv(output_file, index=False)

print("-" * 30)
print(f"SUCESSO! Dataset pronto em: {output_file}")
print("Primeiras 5 linhas:")
print(df_final.head())

------------------------------
SUCESSO! Dataset pronto em: ../data/processed\hydraulic_uci_processed.csv
Primeiras 5 linhas:
   Pressure_1  Pressure_2  Pressure_3  Pressure_4  Pressure_5  Pressure_6  \
0  160.673492  109.466914    1.991475         0.0    9.842170    9.728097   
1  160.603320  109.354890    1.976234         0.0    9.635142    9.529488   
2  160.347720  109.158845    1.972224         0.0    9.530548    9.427949   
3  160.188088  109.064807    1.946575         0.0    9.438827    9.337430   
4  160.000472  108.931434    1.922707         0.0    9.358762    9.260636   

   Volume_Flow_1  Volume_Flow_2  Temperature_1  Temperature_2  Temperature_3  \
0       6.709815      10.304592      35.621983      40.978767      38.471017   
1       6.715315      10.403098      36.676967      41.532767      38.978967   
2       6.718522      10.366250      37.880800      42.442450      39.631950   
3       6.720565      10.302678      38.879050      43.403983      40.403383   
4       6.69